In [19]:
from google.colab import files

uploaded = files.upload()

In [20]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Reference: 
https://colab.research.google.com/drive/13dZVYEOMhXhkXWfvSMVM1TTtUDrT6Aeh?usp=sharing#scrollTo=x0WeP5PREUuy

In [21]:
!pip install transformers==4.0.0

## Dataset Preprocessing

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import numpy as np
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
import seaborn as sns

import torch
import random
import time
import datetime
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
#df = pd.read_csv('/content/cleaned_lyrics.csv', sep=',', names=['artist', 'lyrics'])
df = pd.read_csv("sciryl.csv", delimiter='\t', header=None, names=['sentence_source'], encoding='latin-1')
print(df)
df.dropna(inplace=True) #remove NA values


                                          sentence_source
0       ,up it give to tryin not cats to out goes this...
1       ,"own our hold we help, ya'll need don't we se...
2             ,"ten two the Y2G, the for love... you Who"
3       ,"Moma Big the nor Poppa, see can't niggas Ya'll"
4       ,"E G-I, double B-I, the it's it, check right,...
...                                                   ...
143852         ,Colombia from coke plate the off Snorting
143853      ,bitches white 8 get tryna man middle the Cut
143854                        ,weight the for beam Triple
143855                                ,stove the on Crack
143856                              ,sink the in Chickens

[143857 rows x 1 columns]


In [24]:
class GPT2Dataset(Dataset):

  def __init__(self, txt_list, gpt2_type="gpt2", max_length=768):

    self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>', sep_token='<LYR>') #gpt2-medium
    self.input_ids = []
    self.attn_masks = []
    
    bos_token = '<|startoftext|>'
    lyr_token = '<LYR>'
    eos_token = '<|endoftext|>'
    for lyric in txt_list:
      #encodings_dict = self.tokenizer(bos_token + ' ' + artist + ' ' + lyr_token + ' ' + lyric + ' ' + eos_token, truncation=True, max_length=max_length, padding="max_length")
      encodings_dict = self.tokenizer(bos_token + ' ' + lyric + ' ' + eos_token, truncation=True, max_length=max_length, padding="max_length")

      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    
  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx] 

In [25]:
dataset = GPT2Dataset(df['sentence_source'].tolist(), max_length=768)

# Split into training, validation, and test sets
train_full_size = int(0.9 * len(dataset))
train_size = int(7/9 * train_full_size)
val_size = train_full_size - train_size
test_size = len(dataset) - train_full_size

train_full_dataset, test_dataset = random_split(dataset, [train_full_size, test_size])
train_dataset, val_dataset = random_split(train_full_dataset, [train_size, val_size])

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [26]:
# Create the DataLoaders for our training, validation, and test datasets.
batch_size = 2
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

# For test the order doesn't matter, so we'll just read them sequentially.
test_dataloader = DataLoader(
            test_dataset, # The test samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

## Fine-Tuning

In [27]:
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

# instantiate the model
model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)

# this step is necessary because we added some tokens (bos_token, etc) to the embeddings
# otherwise the tokenizer and model tensors won't match up
model.resize_token_embeddings(len(dataset.tokenizer))

# Tell pytorch to run this model on the GPU.
device = torch.device("cuda")
model.cuda()

# Set the seed value all over the place to make this reproducible.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [28]:
epochs = 3
learning_rate = 5e-5
warmup_steps = 1e2
epsilon = 1e-8

# this produces sample output every 100 steps
sample_every = 100

In [29]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )

In [30]:
# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
# This changes the learning rate as the training loop progresses
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps)

In [31]:
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [32]:
total_t0 = time.time()

training_stats = []

model = model.to(device)

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()        

        outputs = model(  b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]  

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

            model.eval()
            input_context = "<|startoftext|> Look who's back"
            input_ids = dataset.tokenizer(input_context, return_tensors="pt").input_ids
            input_ids = input_ids.to(device)
            sample_outputs = model.generate(
                                    input_ids=input_ids,
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 200,
                                    top_p=0.95, 
                                    num_return_sequences=1
                                )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, dataset.tokenizer.decode(sample_output, skip_special_tokens=True)))
            
            model.train()

        loss.backward()

        optimizer.step()

        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)       
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)
        
        with torch.no_grad():        

            outputs  = model(b_input_ids, 
#                            token_type_ids=None, 
                             attention_mask = b_masks,
                            labels=b_labels)
          
            loss = outputs[0]  
            
        batch_loss = loss.item()
        total_eval_loss += batch_loss        

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)    

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 3 ========
Training...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   100  of  50,350. Loss: 0.17486536502838135.   Elapsed: 0:00:52.
0:  Look who's back lonesome lonesome lonesome I lones lones My My My My My My My My My my My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My My lonesome my lonesome l my my my lonesome l my my my my my lone my lone lones lonesmy l me my lonesmy l my lones my l


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   200  of  50,350. Loss: 0.14869320392608643.   Elapsed: 0:01:47.
0:  Look who's back My My My My the My My I my the My my I I my the my I my the my I my the I my the I my my I my the I my I my I my the I my My the I my the I my my My the I my I my My I my My the My I the I my I my I my I my My the I my I my My My the I my me my I my I my My the I my my I my My the I my My my I my My my my I my my I my I my My the I my my my I my my I my My My My my the my My My the I my my I my I my I my My my I my I my my I my My the my my my I my I my My my My my I my My my My my my my my my my my my I my my I my I my My my I my


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   500  of  50,350. Loss: 0.07310657203197479.   Elapsed: 0:04:32.
0:  Look who's back who's to back who's Back 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   600  of  50,350. Loss: 0.10664233565330505.   Elapsed: 0:05:27.
0:  Look who's back, come to nothing ain't there And" 


KeyboardInterrupt: ignored

## Save Model

In [33]:
import pickle

with open('/content/gdrive/My Drive/training_stats', 'wb') as fp:
    pickle.dump(training_stats, fp)

In [34]:
import os
# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = '/content/gdrive/My Drive/model_save/'

print("Saving model to %s" % output_dir)
# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
dataset.tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))

Saving model to /content/gdrive/My Drive/model_save/


('/content/gdrive/My Drive/model_save/tokenizer_config.json',
 '/content/gdrive/My Drive/model_save/special_tokens_map.json',
 '/content/gdrive/My Drive/model_save/vocab.json',
 '/content/gdrive/My Drive/model_save/merges.txt',
 '/content/gdrive/My Drive/model_save/added_tokens.json')

## Load Model (plus Stats)

In [35]:
import pickle
with open ('/content/gdrive/My Drive/training_stats', 'rb') as fp:
    training_stats = pickle.load(fp)

In [37]:
# Tell pytorch to run this model on the GPU.
device = torch.device("cuda")

# Load a trained model and vocabulary that you have fine-tuned
model = GPT2LMHeadModel.from_pretrained("/content/gdrive/My Drive/model_save/")
tokenizer = GPT2Tokenizer.from_pretrained("/content/gdrive/My Drive/model_save/")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

## Test Dataset

In [ ]:
import math
#========================================
#                  Test
# ========================================

print("")
print("Running Test...")

t0 = time.time()

model.eval()

total_test_loss = 0
nb_test_steps = 0

# Evaluate data for one epoch
for batch in test_dataloader:
    
    b_input_ids = batch[0].to(device)
    b_labels = batch[0].to(device)
    b_masks = batch[1].to(device)
    
    with torch.no_grad():        

        outputs  = model(b_input_ids,  
                          attention_mask = b_masks,
                        labels=b_labels)
      
        loss = outputs[0]  
        
    batch_loss = loss.item()
    total_test_loss += batch_loss        

avg_test_loss = total_test_loss / len(test_dataloader)

ppl = math.exp(avg_test_loss)

test_time = format_time(time.time() - t0)    

print("  Test Loss: {0:.2f}".format(avg_test_loss))
print("  Test Perplexity: {0:.2f}".format(ppl))
print("  Test took: {:}".format(test_time))

## Inference

In [41]:
model.eval()

prompt = "good"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)

print(generated)

sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=50, 
                                max_length = 300,
                                top_p=0.95, 
                                num_return_sequences=3
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[11274]], device='cuda:0')
0: good,ya with play to how me taught they and me with play To 


1: good,"walls the in hoes them see could I way, a in me found You" 


2: good,"too"" it's 'cause back it call you if And" 


